In [1]:
from matdeeplearn.common import data
import os
import torch
import matplotlib.pyplot as plt
from pprint import pprint

from matdeeplearn.preprocessor.helpers import *
from matdeeplearn.preprocessor.transforms import *
from matdeeplearn.common.graph_data import CustomData

from torch_geometric.loader import DataLoader

import ase

%pprint
%load_ext autoreload
%autoreload 2

torch.set_printoptions(profile="full")

/nethome/sbaskaran31/miniconda3/envs/matdeeplearn/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pretty printing has been turned OFF


In [2]:
dataset = data.get_dataset("/nethome/sbaskaran31/projects/Sidharth/hMOF/raw_5k/")

In [3]:
sample = dataset[0]

In [4]:
sample

Data(pos=[41, 3], cell=[3, 3], cell2=[6], y=[1], z=[257], u=[1, 3], edge_index=[1, 2], cell_offsets=[637, 3], structure_id=[1], x=[257, 151], edge_attr=[1, 50], edge_index_rr=[2, 596], edge_attr_rr=[596, 50])

In [5]:
example = CustomData(
    pos=sample.pos,
    cell=sample.cell,
    cell2=sample.cell2,
    y=sample.y,
    z=sample.z,
    u=sample.u,
    x=sample.x,
    edge_index=sample.edge_index,
    edge_attr=sample.edge_attr,
    edge_index_rr=sample.edge_index_rr,
    edge_attr_rr=sample.edge_attr_rr,
    cell_offsets=sample.cell_offsets,
    structure_id=sample.structure_id,
    n_atoms=torch.tensor([sample.x.shape[0]]),
)

In [6]:
example

CustomData(n_atoms=[1], pos=[41, 3], cell=[3, 3], cell2=[6], y=[1], z=[257], u=[1, 3], x=[257, 151], edge_index=[1, 2], edge_attr=[1, 50], edge_index_rr=[2, 596], edge_attr_rr=[596, 50], cell_offsets=[637, 3], structure_id=[1])

In [7]:
r = 5.0
cell = example.cell
pos = example.pos

In [8]:
loader = DataLoader([example], batch_size=1, shuffle=False)

In [9]:
batch = next(iter(loader))

In [10]:
vpos, vz = generate_virtual_nodes(example.cell2, 3.0, "cpu")

/nethome/sbaskaran31/miniconda3/envs/matdeeplearn/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525539683/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [11]:
pos_v = torch.cat([pos, vpos], dim=0)
z_v = torch.cat([example.z, vz], dim=0)

Comparing the ASE primitive_neighbor_list routine with OCP's optimized torch implementation

In [12]:
(
    first_idex_1,
    second_idex_1,
    rij_1,
    rij_vec_1,
    shifts_1,
) = ase.neighborlist.primitive_neighbor_list(
    "ijdDS",
    (True, True, True),
    ase.geometry.complete_cell(cell),
    pos_v.numpy(),
    cutoff=r,
    self_interaction=False,
    use_scaled_positions=False,
)

In [13]:
print(example.n_atoms)

tensor([257])


In [15]:
cell.shape

torch.Size([3, 3])

In [33]:
cell = example.cell
edge_index, cell, neighbors_img = radius_graph_pbc(r, 100, pos_v, cell.view(1, 3, 3), example.n_atoms, "cpu")

In [26]:
from torch_geometric.utils import remove_self_loops

In [35]:
ase_ei = torch.tensor([first_idex_1, second_idex_1])

In [36]:
ocp_ei = edge_index

In [37]:
print(remove_self_loops(ase_ei)[0].shape)
print(remove_self_loops(ocp_ei)[0].shape)

torch.Size([2, 7832])
torch.Size([2, 7832])


In [41]:
sort1, index1 = ase_ei.sort(dim=1)
sort2, index2 = ocp_ei.sort(dim=1)

In [42]:
sort1 == sort2

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, T